In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15254882408895759704, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 17996390694362335366
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time
import math
import os
import pandas as pd
from keras.callbacks import EarlyStopping
import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize
from keras.utils.np_utils import to_categorical
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0


In [5]:
batch_size = 32
img_height, img_width = 128, 128
input_shape = (img_height, img_width, 3)
epochs = 15

In [6]:
train_dir = 'data/train/'
test_dir = 'data/test'

In [7]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    validation_split= 0.3,
    zoom_range=0.2,
    shear_range=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 34909 images belonging to 6 classes.
Found 14958 images belonging to 6 classes.
Found 13578 images belonging to 6 classes.


In [8]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 34909
nb_validation_samples: 14958
nb_test_samples: 13578

predict_size_train: 1091
predict_size_validation: 468
predict_size_test: 425

 num_classes: 6


In [ ]:
# vgg19_weights ="../input/full-keras-pretrained-no-top/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_weights ="../input/full-keras-pretrained-no-top//inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
# vgg16_weights ="../input/full-keras-pretrained-no-top/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet201_weights ="../input/full-keras-pretrained-no-top/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet121_weights ="../input/full-keras-pretrained-no-top/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"
# resenet50_weights ="../input/full-keras-pretrained-no-top/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_resnet_v2_weights ="../input/full-keras-pretrained-no-top/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
# nasnet_weights ="../input/full-keras-pretrained-no-top/nasnet_large_no_top.h5"
# nasnet_mobile_weights ="../input/full-keras-pretrained-no-top/nasnet_mobile_no_top.h5"
# mobilenet_weights ="../input/full-keras-pretrained-no-top/mobilenet_1_0_224_tf_no_top.h5"

In [9]:
from keras.applications.vgg19 import VGG19
# from keras.applications.inception_v3 import InceptionV3
# from keras.applications.vgg16 import VGG16
# from keras.applications.xception import Xception, preprocess_input
# from keras.applications import DenseNet201
# from keras.applications import DenseNet121
# from keras.applications import ResNet50
# from keras.applications.inception_resnet_v2 import InceptionResNetV2
# from keras.applications import NASNetLarge, NASNetMobile
# from keras.applications import MobileNet

bottleneck_final_model=VGG19(weights="imagenet", include_top=False, pooling = "avg")

In [12]:
# os.mkdir("extracted_features")
extracted_features_dir = "extracted_features/"
model_name = "vgg19_descriptors"

In [13]:
bottleneck_features_train = bottleneck_final_model.predict_generator(train_generator, predict_size_train, max_q_size=1, pickle_safe=False)
np.save(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy', bottleneck_features_train)

In [14]:
bottleneck_features_validation = bottleneck_final_model.predict_generator(validation_generator, predict_size_validation)
np.save(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy', bottleneck_features_validation)

In [15]:
bottleneck_features_test = bottleneck_final_model.predict_generator(test_generator, predict_size_test)
np.save(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy', bottleneck_features_test)

In [16]:

import tensorflow as tf
from keras.backend.tensorflow_backend import get_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import set_session

def reset_keras_tf_session():
    """
    this function clears the gpu memory and set the 
    tf session to not use the whole gpu
    """
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

#     config = tf.ConfigProto()
#     config.gpu_options.allow_growth = True
#     set_session(tf.Session(config=config))

reset_keras_tf_session()

In [17]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [18]:
space = {
         'lr': hp.choice('lr',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8, 0.9]),
         'beta_2':hp.choice('beta_2',[0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.999,0.99,0.995]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [19]:
from sklearn.metrics import accuracy_score
def f_nn(params):   
    print ('Parameters testing: ', params)
  
    adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0)
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    model = Sequential()
    # model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.7))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_data, train_labels,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(validation_data, validation_labels),
                        verbose= 2,
                        callbacks=get_callbacks(params))

    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size= batch_size, verbose=1)

    print("Validation Accuracy: {:.4f}%".format(eval_accuracy * 100))
    print("Validation Loss: {}".format(eval_loss))
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    preds = model.predict(test_data)

    predictions = [i.argmax() for i in preds]
    y_true = [i.argmax() for i in test_labels]
#     cm = confusion_matrix(y_pred=predictions, y_true=y_true)

    print('Test Accuracy: {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))

    print("*_*" * 50)
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': eval_loss, 'status': STATUS_OK, 'model': model}

In [20]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
    return callbacks

In [21]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals = 300, trials=trials)
print(best)

Parameters testing:  {'beta_2': 0.9, 'lr': 1e-07, 'beta_1': 0.2}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 4s - loss: 2.2098 - acc: 0.1657 - val_loss: 1.8479 - val_acc: 0.2166
Epoch 2/15
 - 4s - loss: 2.2029 - acc: 0.1645 - val_loss: 1.8346 - val_acc: 0.2168
Epoch 3/15
 - 4s - loss: 2.1797 - acc: 0.1692 - val_loss: 1.8220 - val_acc: 0.2168
Epoch 4/15
 - 4s - loss: 2.1738 - acc: 0.1727 - val_loss: 1.8104 - val_acc: 0.2168
Epoch 5/15
 - 4s - loss: 2.1586 - acc: 0.1741 - val_loss: 1.7995 - val_acc: 0.2168
Epoch 00005: early stopping
14958/14958 [==============================] - 0s 32us/step 
Validation Accuracy: 21.6807%
Validation Loss: 1.7995262816398918
Test Accuracy: 0.2154956547356017
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_2': 0.5, 'lr': 0.01, 'beta_1': 0.3}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 4s

Epoch 2/15
 - 4s - loss: 1.6739 - acc: 0.2892 - val_loss: 1.6437 - val_acc: 0.1690
Epoch 3/15
 - 4s - loss: 1.5769 - acc: 0.3420 - val_loss: 1.6453 - val_acc: 0.2016
Epoch 4/15
 - 4s - loss: 1.5252 - acc: 0.3790 - val_loss: 1.6463 - val_acc: 0.1957
Epoch 5/15
 - 4s - loss: 1.4890 - acc: 0.3998 - val_loss: 1.6463 - val_acc: 0.1958
Epoch 6/15
 - 4s - loss: 1.4643 - acc: 0.4130 - val_loss: 1.6475 - val_acc: 0.1961
Epoch 00006: early stopping
14958/14958 [==============================] - 1s 34us/step
Validation Accuracy: 19.6082%
Validation Loss: 1.6475288942545903
Test Accuracy: 0.3717778759758433
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_2': 0.999, 'lr': 0.1, 'beta_1': 0.8}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 12.2171 - acc: 0.2298 - val_loss: 12.3919 - val_acc: 0.2312
Epoch 2/15
 - 4s - loss: 11.9945 - acc: 0.2294

Parameters testing:  {'beta_2': 0.9, 'lr': 0.001, 'beta_1': 0.7}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 1.3166 - acc: 0.4697 - val_loss: 1.4916 - val_acc: 0.2754
Epoch 2/15
 - 4s - loss: 1.1398 - acc: 0.5414 - val_loss: 1.4502 - val_acc: 0.3412
Epoch 3/15
 - 4s - loss: 1.0971 - acc: 0.5591 - val_loss: 1.4734 - val_acc: 0.3478
Epoch 4/15
 - 4s - loss: 1.0809 - acc: 0.5663 - val_loss: 1.3945 - val_acc: 0.4292
Epoch 5/15
 - 4s - loss: 1.0601 - acc: 0.5729 - val_loss: 1.4397 - val_acc: 0.3242
Epoch 6/15
 - 4s - loss: 1.0503 - acc: 0.5773 - val_loss: 1.4988 - val_acc: 0.3234
Epoch 7/15
 - 4s - loss: 1.0406 - acc: 0.5816 - val_loss: 1.3791 - val_acc: 0.4200
Epoch 00007: early stopping
14958/14958 [==============================] - 1s 37us/step
Validation Accuracy: 41.9976%
Validation Loss: 1.379070732952248
Test Accuracy: 0.5455884519074974
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*

Epoch 10/15
 - 4s - loss: 1.1091 - acc: 0.5745 - val_loss: 1.3878 - val_acc: 0.4444
Epoch 11/15
 - 4s - loss: 1.1155 - acc: 0.5724 - val_loss: 1.3733 - val_acc: 0.4140
Epoch 12/15
 - 4s - loss: 1.1103 - acc: 0.5772 - val_loss: 1.4517 - val_acc: 0.3452
Epoch 13/15
 - 4s - loss: 1.1230 - acc: 0.5748 - val_loss: 1.4428 - val_acc: 0.4563
Epoch 14/15
 - 4s - loss: 1.1124 - acc: 0.5782 - val_loss: 1.4189 - val_acc: 0.3623
Epoch 15/15
 - 4s - loss: 1.1268 - acc: 0.5750 - val_loss: 1.3865 - val_acc: 0.4074
14958/14958 [==============================] - 1s 38us/step
Validation Accuracy: 40.7407%
Validation Loss: 1.3865209376350542
Test Accuracy: 0.5503019590514067
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_2': 0.9, 'lr': 0.01, 'beta_1': 0.7}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 1.4586 - acc: 0.3833 - val_loss: 1.5441 - val_

Epoch 6/15
 - 4s - loss: 1.1429 - acc: 0.5542 - val_loss: 1.4786 - val_acc: 0.2938
Epoch 7/15
 - 4s - loss: 1.1190 - acc: 0.5652 - val_loss: 1.4574 - val_acc: 0.3141
Epoch 8/15
 - 4s - loss: 1.1023 - acc: 0.5694 - val_loss: 1.4913 - val_acc: 0.3019
Epoch 00008: early stopping
14958/14958 [==============================] - 1s 40us/step
Validation Accuracy: 30.1912%
Validation Loss: 1.4912576694223296
Test Accuracy: 0.4765061128295773
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_2': 0.9, 'lr': 1e-06, 'beta_1': 0.8}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 6s - loss: 2.3622 - acc: 0.1442 - val_loss: 1.9369 - val_acc: 0.2302
Epoch 2/15
 - 4s - loss: 2.1883 - acc: 0.1681 - val_loss: 1.8134 - val_acc: 0.2311
Epoch 3/15
 - 4s - loss: 2.0641 - acc: 0.1840 - val_loss: 1.7348 - val_acc: 0.2312
Epoch 4/15
 - 4s - loss: 1.9808 - acc: 0.1997 - 

...

Parameters testing:  {'beta_2': 0.9, 'lr': 0.001, 'beta_1': 0.1}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 20s - loss: 1.3062 - acc: 0.4742 - val_loss: 1.4469 - val_acc: 0.3847
Epoch 2/15
 - 10s - loss: 1.1477 - acc: 0.5366 - val_loss: 1.4283 - val_acc: 0.3367
Epoch 3/15
 - 10s - loss: 1.1059 - acc: 0.5517 - val_loss: 1.4360 - val_acc: 0.3338
Epoch 4/15
 - 11s - loss: 1.0818 - acc: 0.5627 - val_loss: 1.4363 - val_acc: 0.3675
Epoch 00004: early stopping
14958/14958 [==============================] - 2s 137us/step
Validation Accuracy: 36.7496%
Validation Loss: 1.4363261058115675
Test Accuracy: 0.5148033583738401
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_2': 0.5, 'lr': 1e-05, 'beta_1': 0.6}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 21s - loss: 1.8834 - acc: 0.2153 - val_loss: 1.6542 - val_acc: 0.1589
Epoch 2/15

 - 21s - loss: 1.3214 - acc: 0.4680 - val_loss: 1.5173 - val_acc: 0.2666
Epoch 2/15
 - 10s - loss: 1.1485 - acc: 0.5383 - val_loss: 1.4231 - val_acc: 0.4074
Epoch 3/15
 - 11s - loss: 1.1009 - acc: 0.5603 - val_loss: 1.3682 - val_acc: 0.4276
Epoch 4/15
 - 11s - loss: 1.0818 - acc: 0.5660 - val_loss: 1.4708 - val_acc: 0.3368
Epoch 5/15
 - 11s - loss: 1.0602 - acc: 0.5759 - val_loss: 1.4170 - val_acc: 0.3305
Epoch 6/15
 - 11s - loss: 1.0491 - acc: 0.5800 - val_loss: 1.3945 - val_acc: 0.4283
Epoch 7/15
 - 9s - loss: 1.0390 - acc: 0.5830 - val_loss: 1.3942 - val_acc: 0.3806
Epoch 8/15
 - 11s - loss: 1.0272 - acc: 0.5886 - val_loss: 1.3469 - val_acc: 0.4406
Epoch 9/15
 - 11s - loss: 1.0246 - acc: 0.5874 - val_loss: 1.3451 - val_acc: 0.4167
Epoch 10/15
 - 11s - loss: 1.0194 - acc: 0.5899 - val_loss: 1.4258 - val_acc: 0.3427
Epoch 11/15
 - 11s - loss: 1.0079 - acc: 0.5958 - val_loss: 1.3850 - val_acc: 0.3560
Epoch 00011: early stopping
14958/14958 [==============================] - 2s 134us/st

Epoch 4/15
 - 10s - loss: 1.0829 - acc: 0.5665 - val_loss: 1.5024 - val_acc: 0.2942
Epoch 5/15
 - 11s - loss: 1.0605 - acc: 0.5756 - val_loss: 1.4517 - val_acc: 0.3438
Epoch 6/15
 - 9s - loss: 1.0509 - acc: 0.5784 - val_loss: 1.4245 - val_acc: 0.4026
Epoch 7/15
 - 13s - loss: 1.0397 - acc: 0.5795 - val_loss: 1.3971 - val_acc: 0.3714
Epoch 8/15
 - 11s - loss: 1.0343 - acc: 0.5827 - val_loss: 1.3776 - val_acc: 0.3675
Epoch 9/15
 - 11s - loss: 1.0270 - acc: 0.5885 - val_loss: 1.4382 - val_acc: 0.3466
Epoch 00009: early stopping
14958/14958 [==============================] - 2s 129us/step
Validation Accuracy: 34.6637%
Validation Loss: 1.4381733107377954
Test Accuracy: 0.517970245986154
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_2': 0.99, 'lr': 0.001, 'beta_1': 0.7}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 20s - loss: 1.3077 - acc: 0.

Epoch 8/15
 - 10s - loss: 1.1074 - acc: 0.5658 - val_loss: 1.4464 - val_acc: 0.3607
Epoch 9/15
 - 11s - loss: 1.0915 - acc: 0.5733 - val_loss: 1.4384 - val_acc: 0.3922
Epoch 10/15
 - 10s - loss: 1.0792 - acc: 0.5763 - val_loss: 1.4198 - val_acc: 0.3831
Epoch 11/15
 - 10s - loss: 1.0660 - acc: 0.5801 - val_loss: 1.4133 - val_acc: 0.3728
Epoch 12/15
 - 11s - loss: 1.0571 - acc: 0.5843 - val_loss: 1.4121 - val_acc: 0.3986
Epoch 13/15
 - 11s - loss: 1.0496 - acc: 0.5876 - val_loss: 1.4066 - val_acc: 0.4017
Epoch 14/15
 - 10s - loss: 1.0385 - acc: 0.5947 - val_loss: 1.4212 - val_acc: 0.3470
Epoch 15/15
 - 11s - loss: 1.0293 - acc: 0.5926 - val_loss: 1.4304 - val_acc: 0.3607
14958/14958 [==============================] - 2s 151us/step
Validation Accuracy: 36.0743%
Validation Loss: 1.430395633225327
Test Accuracy: 0.5072912063632347
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters 

Epoch 2/15
 - 11s - loss: 1.6815 - acc: 0.2883 - val_loss: 1.6407 - val_acc: 0.1927
Epoch 3/15
 - 11s - loss: 1.5862 - acc: 0.3330 - val_loss: 1.6426 - val_acc: 0.1962
Epoch 4/15
 - 10s - loss: 1.5311 - acc: 0.3690 - val_loss: 1.6499 - val_acc: 0.2068
Epoch 00004: early stopping
14958/14958 [==============================] - 2s 162us/step
Validation Accuracy: 20.6779%
Validation Loss: 1.6499215752256955
Test Accuracy: 0.35933127117395786
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_2': 0.999, 'lr': 0.01, 'beta_1': 0.1}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 22s - loss: 2.2704 - acc: 0.3440 - val_loss: 6.9295 - val_acc: 0.2168
Epoch 2/15
 - 12s - loss: 4.7430 - acc: 0.3241 - val_loss: 6.9834 - val_acc: 0.2168
Epoch 3/15
 - 11s - loss: 4.7806 - acc: 0.3276 - val_loss: 6.9957 - val_acc: 0.2168
Epoch 4/15
 - 10s - loss: 4.7992 - acc: